In [4]:
import os
import argparse

import numpy as np
import json


In [139]:
def print_summary(paths, weathers=[1,3,6,8]):
    """
    We plot the summary of the testing for the set selected weathers.
    We take the raw data and print the way
    it was described on CORL 2017 paper

    """

    # Improve readability by adding a weather dictionary
    weather_name_dict =  {1: 'Clear Noon', 3: 'After Rain Noon',
                          6: 'Heavy Rain Noon', 8: 'Clear Sunset',
                          4: 'Cloudy After Rain', 14: 'Soft Rain Sunset'}

    # First we write the entire dictionary on the benchmark folder.
    metrics_summary_lists = []
    for path_ in paths:
        with open(os.path.join(path_, 'metrics.json'), 'r') as fo:
            metrics_summary = json.load(fo)
            metrics_summary_lists.append(metrics_summary)

    # Second we plot the metrics that are already ready by averaging
    metrics_to_average = [
        'episodes_fully_completed',
        'episodes_completion'
    ]

    # We compute the number  of episodes based on size of average completion
    number_of_episodes = len(
        list(metrics_summary['episodes_fully_completed'].items())[0][1])

    number_of_weathers = len(
        list(metrics_summary['episodes_fully_completed'].items()))
    
    for metric in metrics_to_average:
        if metric == 'episodes_completion':
            print("Average Percentage of Distance to Goal Travelled ")
        else:
            print("Percentage of Successful Episodes")
        print("")
        
        metric_sum_values = np.zeros(number_of_episodes)
        for metric_count, metrics_summary in enumerate(metrics_summary_lists):
            #print('  Metrics setup: ', paths[metric_count].split('/')[-1])
            # metric_count = 0
            values = metrics_summary[metric]
            metric_sum_values_weathers = np.zeros(number_of_episodes)  # 4 different tasks
            for weather, tasks in values.items():  # for testing there is only one weather
                weather = int(float(weather))
                if weather not in set(weathers):
                    print("weather not exiset")
                    break
                
                #print('  Weather: ', weather_name_dict[weather])
                for count, t in enumerate(tasks):
                    # if isinstance(t, np.ndarray) or isinstance(t, list):
                    if t == []:
                        print('    Metric Not Computed')
                    else:
                        if number_of_weathers != 1:
                            print('    ', metric_count,' Task:', count, ' -> ',
                                float(sum(t)) / float(len(t)))
                        metric_sum_values_weathers[count] += \
                            (float(sum(t)) / float(len(t))) * 1.0 / float(
                                number_of_weathers)
            
            #print('  Average Between metrics')
            for i in range(len(metric_sum_values_weathers)):
                #print('    Task ', i, ' -> ', metric_sum_values_weathers[i])
                metric_sum_values[i] += metric_sum_values_weathers[i] * 1.0 / float(len(metrics_summary_lists))
            #print("")

        print('  Average Between metrics')
        for i in range(len(metric_sum_values)):
            print('    Task ', i, ' -> {:0.1f}'.format(metric_sum_values[i]*100))
        print("")

    infraction_metrics = [
        'intersection_otherlane',
        'intersection_offroad',
        'collision_other',
        'collision_vehicles',
        'collision_pedestrians'
    ]

    # We need to collect the total number of kilometers for each task

    for metric in infraction_metrics:
        
        if metric == 'collision_pedestrians':
            print('Avg. Kilometers driven before a collision to a PEDESTRIAN')
        elif metric == 'collision_vehicles':
            print('Avg. Kilometers driven before a collision to a VEHICLE')
        elif metric == 'collision_other':
            print('Avg. Kilometers driven before a collision to a STATIC OBSTACLE')
        elif metric == 'intersection_offroad':
            print('Avg. Kilometers driven before going OUTSIDE OF THE ROAD')
        else:
            print('Avg. Kilometers driven before invading the OPPOSITE LANE')
            
        metric_sum_values = np.zeros(number_of_episodes)        
        summed_driven_kilometers = np.zeros(number_of_episodes)
        for metric_count, metrics_summary in enumerate(metrics_summary_lists):
            values_driven = metrics_summary['driven_kilometers']
            values = metrics_summary[metric]

            # print (zip(values.items(), values_driven.items()))
            for items_metric, items_driven in zip(values.items(),
                                              values_driven.items()):
                weather = items_metric[0]
                tasks = items_metric[1]
                tasks_driven = items_driven[1]
                weather = int(float(weather))
                if weather not in set(weathers):
                    print("weather not exiset")
                    break
                #count = 0
                for (count,t), t_driven in zip(enumerate(tasks), tasks_driven):
                    # if isinstance(t, np.ndarray) or isinstance(t, list):
                    if t == []:
                        print('Metric Not Computed')
                    else:
                        if number_of_weathers != 1:
                            if sum(t) > 0:
                                print('    Task ', count,
                                  ' -> ', t_driven / float(sum(t)))
                            else:
                                print('    Task ', count,
                                  ' -> more than', t_driven)
                        metric_sum_values[count] += float(sum(t))
                        summed_driven_kilometers[count] += t_driven
                        
        #print('  Average Between metrics')
        #for i in range(len(metric_sum_values)):
        #    if metric_sum_values[i] == 0:
        #        print('    Task ', i, ' -> more than ',
        #             summed_driven_kilometers[i])
        #   else:
        #        print('    Task ', i, ' -> ',
        #              summed_driven_kilometers[i] / metric_sum_values[i])
        #print("")
        
        print('  Average Between metrics')
        if metric_sum_values[3] == 0:
            print('    Task ', i, ' -> more than ',summed_driven_kilometers[i])
        else:
            print('    Task ', 3, ' -> {:0.2f}'.format(
            summed_driven_kilometers[3] / metric_sum_values[3]))
        print("")

    print("")
    print("")


In [144]:
path = "/home/tai/ws/ijrr_2018/carla_env/CARLA_0.8.2/PythonClient/imitation_learning/_benchmarks_results/non_croped_policy/raw_testing/single_best/"
#path = "/home/tai/ws/ijrr_2018/carla_env/CARLA_0.8.2/PythonClient/imitation_learning/_benchmarks_results/non_croped_policy/different_methods/domain_best_shift/"

path_list = [os.path.join(path, item) for item in os.listdir(path)]

print_summary(path_list)

Percentage of Successful Episodes

  Average Between metrics
    Task  0  -> 13.3
    Task  1  -> 1.3
    Task  2  -> 0.0
    Task  3  -> 0.0

Average Percentage of Distance to Goal Travelled 

  Average Between metrics
    Task  0  -> 37.8
    Task  1  -> 18.4
    Task  2  -> 7.3
    Task  3  -> 5.0

Avg. Kilometers driven before invading the OPPOSITE LANE
  Average Between metrics
    Task  3  -> 0.23

Avg. Kilometers driven before going OUTSIDE OF THE ROAD
  Average Between metrics
    Task  3  -> 0.21

Avg. Kilometers driven before a collision to a STATIC OBSTACLE
  Average Between metrics
    Task  3  -> 0.14

Avg. Kilometers driven before a collision to a VEHICLE
  Average Between metrics
    Task  3  -> 0.29

Avg. Kilometers driven before a collision to a PEDESTRIAN
  Average Between metrics
    Task  3  -> 2.17





In [ ]:
  infraction_metrics = [
        'collision_pedestrians',
        'collision_vehicles',
        'collision_other',
        'intersection_offroad',
        'intersection_otherlane'

    ]

    # We need to collect the total number of kilometers for each task

    for metric in infraction_metrics:
        values_driven = metrics_summary['driven_kilometers']
        values = metrics_summary[metric]
        metric_sum_values = np.zeros(number_of_episodes)
        summed_driven_kilometers = np.zeros(number_of_episodes)

        if metric == 'collision_pedestrians':
            print('Avg. Kilometers driven before a collision to a PEDESTRIAN')
        elif metric == 'collision_vehicles':
            print('Avg. Kilometers driven before a collision to a VEHICLE')
        elif metric == 'collision_other':
            print('Avg. Kilometers driven before a collision to a STATIC OBSTACLE')
        elif metric == 'intersection_offroad':
            print('Avg. Kilometers driven before going OUTSIDE OF THE ROAD')
        else:
            print('Avg. Kilometers driven before invading the OPPOSITE LANE')

        # print (zip(values.items(), values_driven.items()))
        for items_metric, items_driven in zip(values.items(),
                                              values_driven.items()):
            weather = items_metric[0]
            tasks = items_metric[1]
            tasks_driven = items_driven[1]
            weather = int(float(weather))
            if weather in set(weathers):
                print('  Weather: ', weather_name_dict[weather])
                count = 0
                for t, t_driven in zip(tasks, tasks_driven):
                    # if isinstance(t, np.ndarray) or isinstance(t, list):
                    if t == []:
                        print('Metric Not Computed')
                    else:
                        if sum(t) > 0:
                            print('    Task ', count,
                                  ' -> ', t_driven / float(sum(t)))
                        else:
                            print('    Task ', count,
                                  ' -> more than', t_driven)

                        metric_sum_values[count] += float(sum(t))
                        summed_driven_kilometers[count] += t_driven

                    count += 1
        print('  Average Between Weathers')
        for i in range(len(metric_sum_values)):
            if metric_sum_values[i] == 0:
                print('    Task ', i, ' -> more than ',
                      summed_driven_kilometers[i])
            else:
                print('    Task ', i, ' -> ',
                      summed_driven_kilometers[i] / metric_sum_values[i])
        print("")

    print("")
    print("")
